# "SampleRNN: An Unconditional End-to-End Neural Audio Generation Model" - paper implementation - https://arxiv.org/pdf/1612.07837.pdf

In [13]:


import torch
from model.sample_rnn import SampleRNN
from model.predictor import Predictor
from optim.gradient_clipping import gradient_clipping
from nn.sequence_nll_loss_bits import sequence_nll_loss_bits
from trainer.trainer import Trainer
from datasets.dataloader import DataLoader
from datasets.folder_dataset import FolderDataset
from natsort import natsorted
from functools import reduce
import os
import shutil
import sys
from glob import glob
import re

default_params = {
    'exp': 'TEST',
    'n_rnn': 1,
    'dim': 1024,
    'learn_h0': True,
    'q_levels': 256,
    'seq_len': 1024,
    'weight_norm': True,
    'batch_size': 128,
    'val_frac': 0.1,
    'test_frac': 0.1,
    'keep_old_checkpoints': False,
    'datasets_path': 'datasets',
    'results_path': 'results',
    'epoch_limit': 1000,
    'resume': True,
    'sample_rate': 16000,
    'n_samples': 1,
    'sample_length': 80000,
    'loss_smoothing': 0.99,
    'cuda': False,
    'frame_sizes': 16,
    'dataset': 'example'
    
}

tag_params = [
    'exp', 'frame_sizes', 'n_rnn', 'dim', 'learn_h0', 'q_levels', 'seq_len',
    'batch_size', 'dataset', 'val_frac', 'test_frac'
]


def param_to_string(value):
    if isinstance(value, bool):
        return 'T' if value else 'F'
    elif isinstance(value, list):
        return ','.join(map(param_to_string, value))
    else:
        return str(value)


def make_tag(params):
    return '-'.join(
        key + ':' + param_to_string(params[key])
        for key in tag_params
        if key not in default_params or params[key] != default_params[key]
    )


def setup_results_dir(params):
    def ensure_dir_exists(path):
        if not os.path.exists(path):
            os.makedirs(path)

    tag = make_tag(params)
    results_path = os.path.abspath(params['results_path'])
    ensure_dir_exists(results_path)
    results_path = os.path.join(results_path, tag)
    if not os.path.exists(results_path):
        os.makedirs(results_path)
    elif not params['resume']:
        shutil.rmtree(results_path)
        os.makedirs(results_path)

    for subdir in ['checkpoints', 'samples']:
        ensure_dir_exists(os.path.join(results_path, subdir))

    return results_path


def tee_stdout(log_path):
    log_file = open(log_path, 'a', 1)
    stdout = sys.stdout

    class Tee:

        def write(self, string):
            log_file.write(string)
            stdout.write(string)

        def flush(self):
            log_file.flush()
            stdout.flush()

    sys.stdout = Tee()


def make_data_loader(overlap_len, params):
    path = os.path.join(params['datasets_path'], params['dataset'])

    def data_loader(split_from, split_to, eval):
        dataset = FolderDataset(
            path, overlap_len, params['q_levels'], split_from, split_to
        )
        return DataLoader(
            dataset,
            batch_size=params['batch_size'],
            seq_len=params['seq_len'],
            overlap_len=overlap_len,
            shuffle=(not eval),
            drop_last=(not eval)
        )
    return data_loader


In [ ]:
params = default_params

results_path = setup_results_dir(params)
tee_stdout(os.path.join(results_path, 'log'))

model = SampleRNN(
    frame_sizes=params['frame_sizes'],
    n_rnn=params['n_rnn'],
    dim=params['dim'],
    learn_h0=params['learn_h0'],
    q_levels=params['q_levels'],
    weight_norm=params['weight_norm']
)
predictor = Predictor(model)
if params['cuda']:
    model = model.cuda()
    predictor = predictor.cuda()

optimizer = gradient_clipping(torch.optim.Adam(predictor.parameters()))

data_loader = make_data_loader(model.lookback, params)
test_split = 1 - params['test_frac']
val_split = test_split - params['val_frac']

trainer = Trainer(
    predictor, sequence_nll_loss_bits, optimizer,
    data_loader(0, val_split, eval=False),
    cuda=params['cuda']
)

trainer.run(params['epoch_limit'])
